# Create OMMX Artifact

In [1]:
from datetime import datetime  # When the artifact is created
from dateutil.tz import tzlocal
import uuid    # To generate random tag for testing
import logging # To see the log of pushing artifact
logging.basicConfig(level=logging.INFO)

In [2]:
from ommx.artifact import Artifact, ArtifactBuilder
from ommx.testing import SingleFeasibleLPGenerator, DataType

## Ready `ommx.v1.Instance` and `ommx.v1.Solution` to be packed into artifact

In [3]:
generator = SingleFeasibleLPGenerator(3, DataType.INT)
instance = generator.get_v1_instance()
state = generator.get_v1_state()

In [4]:
# Evaluate the instance for the state
solution = instance.evaluate(state)

In [5]:
# Add annotations
instance.title = "Single feasible LP instance"
instance.created = datetime.now(tzlocal())  # Date is stored as RFC3339 format
solution.parameters = (3, "int")            # Solution can store a set of parameters as a JSON

## Build OMMX artifact

In [6]:
builder = ArtifactBuilder.for_github(
    "Jij-Inc",             # GitHub organization
    "ommx",                # Repository name
    "single_feasible_lp",  # Name of artifact
    str(uuid.uuid4())      # Tag of artifact
)

In [7]:
# Add instance to the artifact
instance_desc = builder.add_instance(instance)

In [8]:
# Add solution with the digest of instance
solution.instance = instance_desc.digest
_desc = builder.add_solution(solution)

In [9]:
# Finish building
artifact = builder.build()

In [10]:
artifact.image_name

'ghcr.io/jij-inc/ommx/single_feasible_lp:f1d3395b-1957-4132-9247-faea2962f299'

## Push artifact to container registry

- This artifact will be pushed to <https://github.com/Jij-Inc/ommx/pkgs/container/ommx%2Fsingle_feasible_lp>
- `push` requires authentication using personal access token (PAT) or `GITHUB_TOKEN` on GitHub Actions. There are two ways for authentication:
  - Run CLI `ommx login` which stores authentication info on your machine.
  - Use environment variables `OMMX_BASIC_AUTH_{DOMAIN,USERNAME,PASSWORD}`. For example, you can push your artifact to GitHub contaier registry from GitHub Actions with following setting:

```yaml
        env:
          OMMX_BASIC_AUTH_DOMAIN: ghcr.io
          OMMX_BASIC_AUTH_USERNAME: ${{ github.actor }}
          OMMX_BASIC_AUTH_PASSWORD: ${{ github.token }}
```

In [11]:
artifact.push()

INFO:ommx.artifact:Pushing: ghcr.io/jij-inc/ommx/single_feasible_lp:f1d3395b-1957-4132-9247-faea2962f299
INFO:ocipkg.distribution.auth:Loaded auth info from: /Users/termoshtt/.ocipkg/config.json
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/c3431c88-924c-4702-a2c9-0a0adc68fa47
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/e54df69e-cc8f-4607-ab2e-6b6365b28e00
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/26f1cfe5-1d82-440a-8c84-1e8342dd9583
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_l

# Pull OMMX Artifact from GitHub Container registry

In [12]:
# Load artifact from local registry, or download from contaier reigstry
artifact = Artifact.load("ghcr.io/jij-inc/ommx/single_feasible_lp:f4c6829d-d80e-4c5b-b70e-6dd0c36c3217")
print(artifact.image_name)

ghcr.io/jij-inc/ommx/single_feasible_lp:f4c6829d-d80e-4c5b-b70e-6dd0c36c3217


In [13]:
for desc in artifact.layers:
    print(desc)

{
  "mediaType": "application/org.ommx.v1.instance",
  "digest": "sha256:6b14d70f79418342221a9d2088c55fa57a2cb4bd1e8174f46571fd1082b982cb",
  "size": 263,
  "annotations": {
    "org.ommx.v1.instance.created": "2024-06-07T03:02:09.190323+00:00",
    "org.ommx.v1.instance.title": "Single feasible LP instance"
  }
}
{
  "mediaType": "application/org.ommx.v1.solution",
  "digest": "sha256:2be0635ff1e644d1cb8a0bfcc8a933b0d0b33588fe3ad9538bc8ae0febefc87a",
  "size": 150,
  "annotations": {
    "org.ommx.v1.solution.parameters": "[3, \"int\"]",
    "org.ommx.v1.solution.instance": "sha256:6b14d70f79418342221a9d2088c55fa57a2cb4bd1e8174f46571fd1082b982cb"
  }
}


In [14]:
desc = artifact.get_layer_descriptor("sha256:6b14d70f79418342221a9d2088c55fa57a2cb4bd1e8174f46571fd1082b982cb")
instance = artifact.get_instance(desc)

In [15]:
instance.title

'Single feasible LP instance'

In [16]:
desc = artifact.get_layer_descriptor("sha256:2be0635ff1e644d1cb8a0bfcc8a933b0d0b33588fe3ad9538bc8ae0febefc87a")
solution = artifact.get_solution(desc)

In [17]:
solution.parameters

[3, 'int']